In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load the dataset
df = pd.read_csv('/content/gdrive/MyDrive/AA/synthetic_health_data1.csv')
df.head()



,Heart_Rate_bpm,Step_Count,Sleep_Duration_hours,Calorie_Burn_kcal,Body_Temperature_C,Blood_Oxygen_Level_percent,Stress_Level,Blood_Pressure,Abnormal
0,53.760097,7574.927550,6.947383,1295.153320,35.913987,91.723435,24.513718,75.645849,1
1,52.085614,7444.609547,5.867383,1235.223479,35.989154,94.184935,22.535464,72.269751,1
2,53.691384,7734.310202,5.608766,1203.664647,35.579244,93.534369,21.690369,70.387152,1
3,55.833596,7906.750462,5.307434,1260.245786,35.581697,91.090751,21.988258,70.640406,1
4,52.303882,7472.463729,5.639678,1282.263528,35.546540,90.560234,24.534449,70.872408,1


In [ ]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Features and target variable
X = df.drop(columns='Abnormal')
y = df['Abnormal']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape for RNN: here we're using a sequence length of 1
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

# Build the RNN model
model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 50)                  │           2,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,001 (11.72 KB)

 Trainable params: 3,001 (11.72 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6934 - loss: 0.6547 - val_accuracy: 0.6415 - val_loss: 0.6631
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6895 - loss: 0.6291 - val_accuracy: 0.6415 - val_loss: 0.6600
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6942 - loss: 0.6213 - val_accuracy: 0.6415 - val_loss: 0.6522
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6578 - loss: 0.6366 - val_accuracy: 0.6415 - val_loss: 0.6424
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6655 - loss: 0.6237 - val_accuracy: 0.6415 - val_loss: 0.6351
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6536 - loss: 0.6341 - val_accuracy: 0.6415 - val_loss: 0.6221
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6819 - loss: 0.5879 - val_accuracy: 0.6415 - val_loss: 0.6104
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6773 - loss: 0.5937 - val_accuracy: 0.6415 - val_loss

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert predictions back to original format
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_binary.flatten()})
print(predictions.head())


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6725 - loss: 0.5447 
Test Loss: 0.5619
Test Accuracy: 0.6482
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
     Actual  Predicted
213       1          1
331       1          1
501       1          1
309       1          1
88        1          1


In [ ]:
model.save('rnn1.h5')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Load the trained RNN model
model = tf.keras.models.load_model('/content/rnn1.h5')  # Update with the path to your saved model

# Define normalization parameters based on provided normal values
min_values = np.array([60, 8000, 7, 1300, 36.5, 95, 26, 80])
max_values = np.array([100, 10000, 9, 2450, 37.5, 100, 75, 120])

# New data
new_data = pd.DataFrame({
    'Heart_Rate_bpm': [56.08769192542913],
    'Step_Count': [7429.270440265984],
    'Sleep_Duration_hours': [6.806314325855423],
    'Calorie_Burn_kcal': [1239.7565126580666],
    'Body_Temperature_C': [35.72503894499821],
    'Blood_Oxygen_Level_percent': [92.49681132785923],
    'Stress_Level': [23.451331609083265],
    'Blood_Pressure': [70.09960965]
})

# Normalize manually
new_data_normalized = (new_data - min_values) / (max_values - min_values)

# Handle any values outside the normalization range (e.g., values below 0 or above 1)
new_data_normalized = np.clip(new_data_normalized, 0, 1)

# Reshape for RNN
new_data_reshaped = new_data_normalized.values.reshape((new_data_normalized.shape[0], 1, new_data_normalized.shape[1]))

# Make predictions
new_predictions = model.predict(new_data_reshaped)
# Convert predictions to binary (assuming a threshold of 0.5 for binary classification)
new_predictions_binary = (new_predictions > 0.5).astype(int)

# Convert predictions to DataFrame
new_data_with_predictions = new_data.copy()
new_data_with_predictions['Predicted_Abnormal'] = new_predictions_binary.flatten()

print(new_data_with_predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
   Heart_Rate_bpm  Step_Count  Sleep_Duration_hours  Calorie_Burn_kcal  \
0       56.087692  7429.27044              6.806314        1239.756513   

   Body_Temperature_C  Blood_Oxygen_Level_percent  Stress_Level  \
0           35.725039                   92.496811     23.451332   

   Blood_Pressure  Predicted_Abnormal  
0        70.09961                   1  
